In [1]:
# import modules
import pandas as pd
import numpy as np
import subprocess
import re

In [2]:
# optuna_study.db에서 best parameter set 불러오기
import sqlite3
import optuna

# optuna db 연결
storage_name = "sqlite:///optuna_study.db" 
study = optuna.load_study(study_name="parameter_optimization", storage=storage_name)

# best trial의 parameters 가져오기
best_params = study.best_trial.params

# best parameters 출력
print(f"Best parameters from Optuna study:")
for param_name, value in best_params.items():
    print(f"{param_name}: {value}")


c:\Users\USER\.conda\envs\plaskin_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Best parameters from Optuna study:
p0: -0.14912943544085183
p1: -0.15392556229577198
p2: -0.163931365896338
p3: 0.1717249571593621
p4: -0.10272662641007166
p5: -0.08956837581720675
p6: 0.21686296715861408
p7: 0.08730398028714918
p8: 0.15881420911984612
p9: 0.1634334895972859
p10: 0.04558638656683429
p11: -0.15535304320577015
p12: -0.07054618214459477
p13: -0.042765006225964255
p14: -0.18208555109599392
p15: -0.25591398803058424
p16: -0.06227841019209614
p17: 0.2482379018703084
p18: 0.0046191201429280765
p19: 0.06598008315311601
p20: 0.0712427563916892
p21: -0.09631934963985524
p22: -0.14598794459130582
p23: -0.008542871413253413
p24: 0.15172407094904142
p25: 0.10697101941755181
p26: -0.039863119338715496
p27: 0.07005947622988978
p28: 0.028699824044033966
p29: -0.1314786427807285
p30: -0.011762941849380627
p31: -0.03933273210407419


In [3]:
def modify_parameter(param_name: str, new_value: float, Voltage: float) -> None:
    # modify the parameter value in the kinet.inp file
    with open(f'kinet{Voltage}_opt.inp', 'r') as f:
        content = f.read()

    # modify the parameter
    pattern = rf'(parameter :: {param_name} = )([\d\.\+\-d]+)'
    new_value_str = f'{new_value:.4e}'.replace('e', 'd')
    content = re.sub(pattern, f'parameter :: {param_name} = {new_value_str}', content)
    
    with open(f'kinet{Voltage}_opt.inp', 'w') as f:
        f.write(content)

In [4]:
def run_prep(Voltage):
    kinet_path = f'kinet{Voltage}_opt.inp'
    process = subprocess.Popen(
        './preprocessor.exe',
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=False)
    
    process.stdin.write(f'{kinet_path}\n'.encode())
    process.stdin.flush()

    process.stdin.write('.\n'.encode())
    process.stdin.flush()

    output, error = process.communicate()
    output_str = output.decode('utf-8', errors='ignore') if output else ''
    error_str = error.decode('utf-8', errors='ignore') if error else ''

    return output_str, error_str

def compile_zdp(Voltage):
    exe_path = f'run_{Voltage}kV.exe'
    compile_command= [
        'gfortran', '-o', exe_path, 'dvode_f90_m.F90', 'zdplaskin_m.F90', f'run_{Voltage}kV.F90', 'bolsig_x86_64_g.dll'
    ]
    result = subprocess.run(compile_command, capture_output=True, text=True)

    if result.returncode != 0:
        raise Exception(f"{exe_path} 컴파일 실패")

def run_zdplaskin(Voltage):
    exe_path = f'run_{Voltage}kV.exe'
    try:
        process = subprocess.Popen(
            exe_path,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            stdin=subprocess.PIPE,
            universal_newlines=True,
            bufsize=1,
        )

        while True:
            output = process.stdout.readline()

            if not output:
                break
            print(f'\r{output.strip()}                                                                              ',end='',flush=True)

            if "PRESS ENTER TO EXIT" in output:
                process.kill()
                break

            if "WARNING: BOLSIG+ convergence failed" in output:
                process.stdin.write('\n')
                process.stdin.flush()

    except:
        pass
    return process

def err_cal(Voltage):
    #read the result file
    species = []
    with open('qt_species_list.txt', 'r') as f:
        for line in f:
            comp = line[2:]
            species.append(comp.strip())
    
    df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
    
    # calculate the concentration
    H2 = (df_sp['H2'])
    CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])
    C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V2)'] + df_sp['C2H2(V5)'] + df_sp['C2H2(V13)'])
    C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)'] + df_sp['C2H4(V2)'])
    C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)'] + df_sp['C2H6(V24)'])
    C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])
    C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])
    C4H10 = (df_sp['C4H9H'])
    C5H12 = (df_sp['C5H12'])
    C = (df_sp['C'])
    CH2 = (df_sp['CH2'])
    CH3 = (df_sp['CH3'])
    C2H3 = (df_sp['C2H3'])
    C2H5 = (df_sp['C2H5'])
    C3H7 = (df_sp['C3H7'])
    H = (df_sp['H'])
    CH3_plus = (df_sp['CH3^+'])
    CH4_plus = df_sp['CH4^+']
    CH5_plus = df_sp['CH5^+']
    C2H2_plus = df_sp['C2H2^+']
    C2H4_plus = df_sp['C2H4^+']
    C2H5_plus = df_sp['C2H5^+']
    C2H6_plus = df_sp['C2H6^+']
    C3H6_plus = df_sp['C3H6^+']
    C3H8_plus = df_sp['C3H8^+']

    t = abs(df_sp['Time [s]']-16.9646).argmin()

    sim_H2 = float(format(H2.iloc[t], '.6f'))
    sim_CH4 = float(format(CH4.iloc[t], '.6f'))
    sim_C2H2 = float(format(C2H2.iloc[t], '.6f'))
    sim_C2H4 = float(format(C2H4.iloc[t], '.6f'))
    sim_C2H6 = float(format(C2H6.iloc[t], '.6f'))
    sim_C3H6 = float(format(C3H6.iloc[t], '.6f'))
    sim_C3H8 = float(format(C3H8.iloc[t], '.6f'))
    sim_C4H10 = float(format(C4H10.iloc[t], '.6f'))
    sim_C5H12 = float(format(C5H12.iloc[t], '.6f'))
    sim_C = float(format(C.iloc[t], '.6f'))
    sim_CH2 = float(format(CH2.iloc[t], '.6f'))
    sim_CH3 = float(format(CH3.iloc[t], '.6f'))
    sim_C2H3 = float(format(C2H3.iloc[t], '.6f'))
    sim_C2H5 = float(format(C2H5.iloc[t], '.6f'))
    sim_C3H7 = float(format(C3H7.iloc[t], '.6f'))
    sim_H = float(format(H.iloc[t], '.6f'))
    sim_CH3_plus = float(format(CH3_plus.iloc[t], '.6f'))
    sim_CH4_plus = float(format(CH4_plus.iloc[t], '.6f'))
    sim_CH5_plus = float(format(CH5_plus.iloc[t], '.6f'))
    sim_C2H2_plus = float(format(C2H2_plus.iloc[t], '.6f'))
    sim_C2H4_plus = float(format(C2H4_plus.iloc[t], '.6f'))
    sim_C2H5_plus = float(format(C2H5_plus.iloc[t], '.6f'))
    sim_C2H6_plus = float(format(C2H6_plus.iloc[t], '.6f'))
    sim_C3H6_plus = float(format(C3H6_plus.iloc[t], '.6f'))
    sim_C3H8_plus = float(format(C3H8_plus.iloc[t], '.6f'))

    out_H2 = sim_H2 - sim_CH2 - 0.5*sim_CH3 - 0.5*sim_CH3_plus + 0.5*sim_CH5_plus - 0.5*sim_C2H3 + 0.5*sim_C2H5 + 0.5*sim_C2H5_plus + 0.5*sim_C3H7 + 0.5*sim_H
    out_CH4 = sim_CH4 + sim_CH2 + sim_CH3 + sim_CH3_plus + sim_CH4_plus + sim_CH5_plus
    out_C2H6 = sim_C2H6 + sim_C2H6_plus
    out_C2H4 = sim_C2H4 + sim_C2H3 + sim_C2H5 + sim_C2H4_plus + sim_C2H5_plus
    out_C2H2 = sim_C2H2 + sim_C2H2_plus
    out_C3H8 = sim_C3H8 + sim_C3H8_plus
    out_C3H6 = sim_C3H6 + sim_C3H7 + sim_C3H6_plus
    out_C4H10 = sim_C4H10
    out_C5H12 = sim_C5H12
    out_C = sim_C
    out_total = out_H2 + out_CH4 + out_C2H6 + out_C2H4 + out_C2H2 + out_C3H8 + out_C3H6 + out_C4H10 + out_C5H12 + out_C
    newsim = [out_H2/out_total*100, out_CH4/out_total*100, out_C2H6/out_total*100, out_C2H4/out_total*100, out_C2H2/out_total*100, out_C3H8/out_total*100, out_C3H6/out_total*100, out_C4H10/out_total*100, out_C5H12/out_total*100, out_C/out_total*100]
    
    return newsim

In [5]:
PD0 = 4.317
volt = [10, 12.5, 15]
res = []
for v in volt:
    if v == 10:
        PD = 2.64
    elif v == 12.5:
        PD = 3.273
    else:
        PD = PD0
    
    para = best_params.copy()
    for para_name in para.keys():
        para[para_name] = np.exp(best_params[para_name]*(PD0-PD))

    for para_name, value in para.items():
        modify_parameter(para_name, value, v)
    
    run_prep(v)
    compile_zdp(v)
    run_zdplaskin(v)

    sim = err_cal(v)
    res.append(sim)


PRESS ENTER TO EXIT ...At line 89 of file run_15kV.F90 (unit = 5, file = 'fort.5')                                                                                     

In [6]:
# load kinetic data
df_kinetic = pd.read_csv('kinetic_data.csv')
df_mol = pd.read_csv('kinetic_data.csv').filter(like='mol%')
df_power = pd.read_csv('kinetic_data.csv').filter(like='Power')

In [7]:
exp = df_mol.T
df_res = exp.copy()
df_res.columns = ['15kV_exp', '12.5kV_exp', '10kV_exp']
df_res['10kV_sim'] = res[0]
df_res['12.5kV_sim'] = res[1]
df_res['15kV_sim'] = res[2]
# 열 순서를 15kV, 12.5kV, 10kV 순서로 변경
df_res = df_res[['10kV_exp','10kV_sim', '12.5kV_exp', '12.5kV_sim', '15kV_exp', '15kV_sim']]








In [8]:
df_res

,10kV_exp,10kV_sim,12.5kV_exp,12.5kV_sim,15kV_exp,15kV_sim
mol%H2,3.716800,4.354385,6.589786,5.703115,9.498213,8.429129
mol%CH4,94.052085,92.990048,89.513818,90.813104,84.919569,86.462121
mol%C2H6,1.129639,1.211618,1.847889,1.646326,2.574999,2.498329
mol%C2H4,0.119020,0.063148,0.147363,0.092722,0.176056,0.173367
mol%C2H2,0.082875,0.131371,0.109703,0.172873,0.136863,0.238285
mol%C3H8,0.292595,0.409574,0.529731,0.571181,0.769792,0.883177
mol%C3H6,0.019609,0.048021,0.030921,0.073785,0.042373,0.136740
mol%C4H10,0.037305,0.041752,0.079794,0.066049,0.122806,0.128792
mol%C5H12,0.046339,0.016774,0.088350,0.034693,0.130879,0.103076
mol%C,0.579605,0.733310,1.186528,0.826152,1.800937,0.946984
